In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

## Step 1: Load data

In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

# for convenience
datasets = [df_train, df_test]

## Step 2: Basic cleaning and preprocessing

In [3]:
# Fill in missing values
mAge = pd.concat((df_train['Age'], df_test['Age']), axis=0).mean()
medFare = pd.concat((df_train['Fare'], df_test['Fare']), axis=0).median()
for df in datasets:
    df['Age'] = df['Age'].fillna(mAge)
    df['Fare'] = df['Fare'].fillna(medFare)
    df['Embarked'] = df['Embarked'].fillna('S')

## Step 3: Feature engineering

In [4]:
# find fare bins based on frequency rather than value
garbage, fare_bins = pd.qcut(df_train['Fare'].append(df_test['Fare']), 4, retbins=True)
# loops through both train and test set for convenience
for df in datasets:
    # bin age
    df['Age_binned'] = pd.cut(df['Age'], [0,16,32,48,64,200], labels = [0,1,2,3,4], retbins=False)
    
    # bin fare
    df['Fare_binned'] = pd.cut(df['Fare'], fare_bins, labels = [0,1,2,3], include_lowest=True, retbins=False)
    
    # family features
    df['Family_size'] = df_train['SibSp'] + df_train['Parch']
    df['Is_Alone'] = (df['Family_size'] == 0).astype(int)

In [5]:
# drop unneeded rows
for df in datasets:
    df.drop(['Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Name', 'Fare_binned'], axis=1, inplace=True)
    
# need to keep passengerID for submission

## Step 4: Format data for modelling

In [6]:
# label encode categoricals 
le = LabelEncoder()
le.fit(df_train['Sex'])
df_train.loc[:,'Sex'] = le.transform(df_train['Sex'])
df_test.loc[:,'Sex'] = le.transform(df_test['Sex'])

le.fit(df_train['Embarked'])
df_train['Embarked'] = le.transform(df_train['Embarked'])
df_test['Embarked'] = le.transform(df_test['Embarked'])

In [7]:
# Split data into X and y, and select features to use
X = df_train.drop(['Survived', 'PassengerId'], axis=1)
y = df_train['Survived']
X_test = df_test.drop(['PassengerId'], axis=1)

In [8]:
# Split data X and y into train and val sets
X_train, X_val, y_train, y_val = train_test_split(
      X, y, test_size=0.2, random_state=42)

In [9]:
df_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Embarked       0
Age_binned     0
Family_size    0
Is_Alone       0
dtype: int64

## Step 5: Train (or fit) the model

In [22]:
# Fit logistic regression
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
# fit rf
rf = RandomForestClassifier(random_state=12)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

## Step 6: Evaluate the model

In [24]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [25]:
# Use model to predict on train and val sets
y_pred_trn_lr = lr.predict(X_train)
y_pred_val_lr = lr.predict(X_val)

y_pred_trn_rf = rf.predict(X_train)
y_pred_val_rf = rf.predict(X_val)

In [26]:
print('LR train set accuracy', acc(y_train, y_pred_trn_lr))
print('LR val set accuracy', acc(y_val, y_pred_val_lr))
print('RF train set accuracy', acc(y_train, y_pred_trn_rf))
print('RF val set accuracy', acc(y_val, y_pred_val_rf))

LR train set accuracy 0.8047752808988764
LR val set accuracy 0.8044692737430168
RF train set accuracy 0.851123595505618
RF val set accuracy 0.7932960893854749


## Step 7: Generate Submission

In [15]:
# fit on whole dataset
lr.fit(X, y)

# Predict for test set
y_pred_test = lr.predict(X_test)

# Create a Kaggle submission
sub = pd.DataFrame({'PassengerId': df_test['PassengerId'],
                    'Survived': y_pred_test})

sub.to_csv('week_3_baseline.csv', index=False)